In [1]:
#Import the libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import time
import requests

In [2]:
#My login credentials
login_email = 'dhirajkhanna@gmail.com'
login_password = 'dhiraj0401'

In [3]:
# open chromedriver
driver = webdriver.Chrome('/Users/matthewroumelis/Downloads/chromedriver')
time.sleep(2)

In [4]:
# navigate to login page
driver.get('https://database.coffeeinstitute.org/login')
time.sleep(2)

In [5]:
# submit login credentials 
username = driver.find_element_by_xpath("//input[@name = 'username']")
password = driver.find_element_by_xpath("//input[@name = 'password']")
time.sleep(2)

In [6]:
username.send_keys(login_email)
password.send_keys(login_password)
driver.find_element_by_xpath("//input[@type = 'submit']").click()
time.sleep(2)

In [7]:
# navigate to coffees page, then to arabicas page containing links to all quality reports 
coffees = driver.find_element_by_xpath("//a[@class='coffees']").click()
time.sleep(3)
driver.find_element_by_xpath('//a[@class = "btn arabica"]').click() #btn robusta
time.sleep(3)

In [8]:
# Find hrefs for all individual coffees on page 1
elems = driver.find_elements_by_xpath("//td/a[contains(text(), '#')]")
coffee_ids = [elem.get_attribute('href') for elem in elems]

In [9]:
#Do the same for the other two pages (Robusta has only one page so you don't need to go beyond one page for Robusta)
#First click on the next page button
driver.find_element_by_xpath("//a[@class = 'paginate_button next']").click()

In [10]:
elems = driver.find_elements_by_xpath("//td/a[contains(text(), '#')]")
coffee_ids.append([elem.get_attribute('href') for elem in elems])

In [11]:
#Now go to the last page and do the same
driver.find_element_by_xpath("//a[@class = 'paginate_button next']").click()
elems = driver.find_elements_by_xpath("//td/a[contains(text(), '#')]")
coffee_ids.append([elem.get_attribute('href') for elem in elems])

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472.77)


In [ ]:
print(len(coffee_ids))

In [12]:
coffee_ids = list(pd.core.common.flatten(coffee_ids))

In [13]:
print(len(coffee_ids))

100


In [14]:
coffeenum = 0
for id in coffee_ids:
    driver.get(id)
    time.sleep(3)
#     print(id)
    tables = driver.find_elements(By.TAG_NAME, "table")
    j = 0
    for tab in tables:
        try:
            t = BeautifulSoup(tab.get_attribute('outerHTML'), "html.parser")
            df = pd.read_html(str(t))
            name = 'coffee_{}_table_{}.csv'.format(coffeenum,j)
            df[0].to_csv(name)
#             print(name)
        except:
            print('ERROR: {} failed'.format(name))
            print('ERROR: {} failed'.format(name))
            print('ERROR: {} failed'.format(name))
            print('ERROR: {} failed'.format(name))
            print('ERROR: {} failed'.format(name))
        j += 1
    coffeenum += 1

In [ ]:
df_list = []
for i in range(10):
    if(os.path.isfile('coffee_{}_table_1.csv'.format(i))):
        df1 = pd.read_csv('coffee_{}_table_1.csv'.format(i))
    if(os.path.isfile('coffee_{}_table_2.csv'.format(i))):
        df2 = pd.read_csv('coffee_{}_table_2.csv'.format(i))
    if(os.path.isfile('coffee_{}_table_3.csv'.format(i))):
        df3 = pd.read_csv('coffee_{}_table_3.csv'.format(i)) 
    if(os.path.isfile('coffee_{}_table_4.csv'.format(i))):
        df4 = pd.read_csv('coffee_{}_table_4.csv'.format(i))
    
    #For the first table
    df1.columns = ['one','two','three','four','five']
    colnames1 = df1['two'].tolist()
    colnames2 = df1['four'].tolist()
    data1 = df1['three'].tolist()
    data2 = df1['five'].tolist()
    df1_processed = pd.DataFrame([(data1+data2)],columns=(colnames1+colnames2))
    
    #For the second table
    df2.columns = ['one','two','three','four','five']
    colnames1 = df2['two'].tolist()
    colnames2 = df2['four'].tolist()
    data1 = df2['three'].tolist()
    data2 = df2['five'].tolist()
    df2_processed = pd.DataFrame([(data1+data2)],columns=(colnames1+colnames2))
    
    #For the third table
    df3.columns = ['one','two','three','four','five']
    colnames1 = df3['two'].tolist()
    colnames2 = df3['four'].tolist()
    data1 = df3['three'].tolist()
    data2 = df3['five'].tolist()
    df3_processed = pd.DataFrame([(data1+data2)],columns=(colnames1+colnames2))
    
    #For the 4th table
    df4.columns = ['one','two','three']
    colnames1 = df4['two'].tolist()
    data1 = df4['three'].tolist()
    df4_processed = pd.DataFrame([data1],columns=colnames1)
    
    df = pd.concat([df1_processed,df2_processed,df3_processed,df4_processed],1)
    df = df.rename(columns = {np.nan : "NA"})
    df_list.append(df)

In [ ]:
df_final = pd.concat(df_list, 0)

In [ ]:
print(df_final.columns)
print(df_final.shape)
print(df_final.head())

In [ ]:
df_final.to_csv('arabica.csv')